In [1]:
!pip install -qq google-play-scraper

In [2]:
!pip install -qq -U watermark

     |████████████████████████████████| 1.6 MB 7.0 MB/s 


In [3]:
%reload_ext watermark
%watermark -v -p pandas,matplotlib,seaborn,google_play_scraper

Python implementation: CPython
Python version       : 3.7.15
IPython version      : 7.9.0

pandas             : 1.3.5
matplotlib         : 3.2.2
seaborn            : 0.11.2
google_play_scraper: 1.2.2



In [4]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [5]:
app_packages = [
  'jp.konami.pesam'
]

## Scraping App Information


In [6]:
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us') # Specify the language and region
  del info['comments']
  app_infos.append(info)

100%|██████████| 1/1 [00:00<00:00,  4.21it/s]


In [7]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [8]:
print_json(app_infos[0])

{
  "adSupported": true,
  "appId": "jp.konami.pesam",
  "containsAds": true,
  "contentRating": "Everyone",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "Approximately 3.4 GB of free space will be required to install this update, so please confirm that you have enough room on your device before commencing the download.\r\n*Given the size of the file, we highly recommend that you connect to a Wi-Fi network to download it.\r\n\r\nWe have received reports from some users indicating that Google Play does not always display the correct download size.\r\n\r\nTherefore we urge you to wait a while even after the download progress bar has reached 100% to ensure that your device has finished downloading the complete file.\r\nAlso, you will not be able to complete the update if your device does not have enough space to install the file.\r\nIf the app fails to boot, even if the download progress bar has reached 100% in the Google Play Store, please wait a little while

In [9]:
def format_title(title):
  sep_index = title.find(':') if title.find(':') != -1 else title.find('-')
  if sep_index != -1:
    title = title[:sep_index]
  return title[:10]

fig, axs = plt.subplots(2, len(app_infos) // 2, figsize=(14, 5))

for i, ax in enumerate(axs.flat):
  ai = app_infos[i]
  img = plt.imread(ai['icon'])
  ax.imshow(img)
  ax.set_title(format_title(ai['title']))
  ax.axis('off')

<Figure size 1008x360 with 0 Axes>

In [10]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

## Scraping App Reviews


In [11]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 200 if score == 3 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 1/1 [00:03<00:00,  3.22s/it]


In [12]:
print_json(app_reviews[0])

{
  "appId": "jp.konami.pesam",
  "at": "2022-10-20 10:17:14",
  "content": "The style of play has become very slow now, the defense performs as the spectator is, there is no automatic switch between the players, the contracts of the legendary players are few and very expensive and illogical for me, in addition to the problem of changing the places of the control buttons and this made me unable to control because of changing the places of the buttons and I became a spectator of the matches due to my inability to control, i deleted it cuz i don't longer like it.",
  "repliedAt": null,
  "replyContent": null,
  "reviewCreatedVersion": "7.1.1",
  "reviewId": "ffc34bcd-e701-4efd-821c-301433866f58",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 32,
  "userImage": "https://play-lh.googleusercontent.com/a-/ACNPEu9sNTJ_G30PmJhJftGaBst0oGqRBPMI2K_UZHBDhA",
  "userName": "\u0644\u0648\u0631\u062f - LORD"
}



In [13]:
len(app_reviews)

1200

In [14]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviews.csv', index=None, header=True)